In [1]:
%pip install datasets
%pip install transformers
%pip install evaluate
%pip install nltk


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.



Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import Dataset, load_dataset
from evaluate import load as load_metric
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import EarlyStoppingCallback

import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
import json
import time



In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())

1
NVIDIA GeForce RTX 3070
0


In [ ]:

# Pariya File Paths
# train_file_path = "/content/drive/MyDrive/nlp_project/train.jsonlist"
# validation_file_path = "/content/drive/MyDrive/nlp_project/validation.jsonlist"
# test_file_path = "/content/drive/MyDrive/nlp_project/test.jsonlist"

# Ellie File Paths
train_file_path = r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\train.jsonlist"
validation_file_path = r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\validation.jsonlist"
test_file_path = r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\test.jsonlist"

# Function to process JSON lines to Pandas DataFrame
import json

# Add your normalization function here
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the NLTK lemmatizer
lemmatizer = WordNetLemmatizer()

# Set up stopwords list
stop_words = set(stopwords.words("english"))

# Define the normalization function
def normalize_text(text):
    # 1. Convert text to lowercase
    text = text.lower()

    # 2. Remove non-alphabetic characters (punctuation, numbers, etc.)
    text = re.sub(r'[^a-z\s]', '', text)

    # 3. Tokenize the text (split into words)
    tokens = nltk.word_tokenize(text)

    # 4. Remove stopwords (optional)
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatization (optional)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 6. Join tokens back into a string
    return ' '.join(tokens)

def json_to_dataset(path):
    dataset = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                l = json.loads(line.strip())  # Safely parse JSON lines
                newL = {
                    "labels": 1 if l["verdict"] == "not the asshole" else 0,
                    "text": l["body"]
                }
                dataset.append(newL)
            except json.JSONDecodeError as e:
                print(f"Skipping line due to JSON error: {e}")
                print(line)
                continue  # Skip invalid lines
            except KeyError as e:
                print(f"Skipping line due to missing key: {e}")
                continue  # Skip lines with missing keys
    return Dataset.from_pandas(pd.DataFrame(dataset))


# Load datasets
dataset_train = json_to_dataset(train_file_path)
dataset_validation = json_to_dataset(validation_file_path)
dataset_test = json_to_dataset(test_file_path)


print(dataset_train)
print(dataset_validation)
print(dataset_test)

Dataset({
    features: ['labels', 'text'],
    num_rows: 32000
})
Dataset({
    features: ['labels', 'text'],
    num_rows: 4000
})
Dataset({
    features: ['labels', 'text'],
    num_rows: 4000
})


In [5]:
# Load Tokenizer
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization Function
def tokenize_function(examples):
	return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize datasets
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_validation = dataset_validation.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

building tokenizer...


Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
# Load Model
id2label = {0: "asshole", 1: "not the asshole"}
label2id = {"asshole": 0, "not the asshole": 1}
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2, id2label=id2label, label2id=label2id)
model.cuda()
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir= r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\ModelOutput",
    num_train_epochs=2,  # Train for fewer epochs
    per_device_train_batch_size=32,  # Larger batch size
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,  # Regularization
    warmup_steps=500,  # Gradual learning rate warmup
    lr_scheduler_type="cosine",  # Use cosine decay for learning rate
    save_total_limit=1,  # Save only the best model
    load_best_model_at_end=True,  # Load the best model at the end
    logging_dir='./logs',
    logging_steps=10,
)




Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
print(torch.cuda.is_available())

True


In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    print("Labels:", labels)
    print("Predictions:", predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary"
    )
    accuracy = accuracy_score(labels, predictions)

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

metrics = compute_metrics((np.array([[0.8, 0.2], [0.1, 0.9]]), np.array([0, 1])))

print("Accuracy:", metrics["accuracy"])
print("F1 Score:", metrics["f1"])
print("Precision:", metrics["precision"])
print("Recall:", metrics["recall"])

Labels: [0 1]
Predictions: [0 1]
Accuracy: 1.0
F1 Score: 1.0
Precision: 1.0
Recall: 1.0


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Early stopping
)


trainer.train()

# Evaluate model on the test set
test_results = trainer.evaluate(tokenized_dataset_test)
print(f"Test set results: {test_results}")

# Save the model and tokenizer
model.save_pretrained(r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\ModelOutput\model")
tokenizer.save_pretrained(r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\ModelOutput\tokenizer")


C:\Users\aeadu\AppData\Local\Temp\ipykernel_11892\3286009903.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/4000 [00:00<?, ?it/s]

{'loss': 0.6998, 'grad_norm': 0.9926924109458923, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.01}
{'loss': 0.7005, 'grad_norm': 0.733194887638092, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.02}
{'loss': 0.7015, 'grad_norm': 0.700109601020813, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.03}
{'loss': 0.6934, 'grad_norm': 0.9348874688148499, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.04}
{'loss': 0.6973, 'grad_norm': 0.6521177291870117, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.05}
{'loss': 0.6948, 'grad_norm': 0.9866229891777039, 'learning_rate': 2.4000000000000003e-06, 'epoch': 0.06}
{'loss': 0.6959, 'grad_norm': 0.6587552428245544, 'learning_rate': 2.8000000000000003e-06, 'epoch': 0.07}
{'loss': 0.7039, 'grad_norm': 0.8565369248390198, 'learning_rate': 3.2000000000000003e-06, 'epoch': 0.08}
{'loss': 0.6898, 'grad_norm': 0.6094939112663269, 'learning_rate': 3.6000000000000003e-06, 'epoch': 0.09}
{'loss': 0.6959, 'grad_norm': 0.6675586700439453,

  0%|          | 0/32 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 0 1 ... 1 0 0]
{'eval_loss': 0.6635096669197083, 'eval_accuracy': 0.60025, 'eval_f1': 0.6074146820525411, 'eval_precision': 0.5967197298601061, 'eval_recall': 0.6185, 'eval_runtime': 562.9447, 'eval_samples_per_second': 7.105, 'eval_steps_per_second': 0.057, 'epoch': 1.0}
{'loss': 0.6148, 'grad_norm': 1.821257472038269, 'learning_rate': 1.897038094386969e-05, 'epoch': 1.01}
{'loss': 0.6048, 'grad_norm': 2.5745089054107666, 'learning_rate': 1.8930350486128855e-05, 'epoch': 1.02}
{'loss': 0.6158, 'grad_norm': 3.8775594234466553, 'learning_rate': 1.8889600530961935e-05, 'epoch': 1.03}
{'loss': 0.6094, 'grad_norm': 2.0009167194366455, 'learning_rate': 1.8848134361497385e-05, 'epoch': 1.04}
{'loss': 0.6145, 'grad_norm': 2.474684476852417, 'learning_rate': 1.880595531856738e-05, 'epoch': 1.05}
{'loss': 0.5973, 'grad_norm': 3.0398943424224854, 'learning_rate': 1.8763066800438638e-05, 'epoch': 1.06}
{'loss': 0.6291, 'grad_norm': 2.642702341079712, 'lea

KeyboardInterrupt: 